<a href="https://colab.research.google.com/github/YugantGotmare/Machine-Learning-Projects/blob/master/Crunchyroll_Animes_and_Movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#  importing libaries
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Loading dataset
anime_data = pd.read_csv('/content/titles.csv')

In [3]:
#  first 5 data from datset
anime_data.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,index
0,ts21930,Lupin the Third,SHOW,Arsene Lupin III is the grandson of the master...,1971,TV-14,23,"['scifi', 'animation', 'action', 'comedy', 'cr...",['JP'],6.0,tt0159175,7.9,2116.0,45.829,8.0,1
1,ts25177,Mobile Suit Gundam,SHOW,Mobile Suit Gundam is a televised anime series...,1979,TV-Y7,25,"['scifi', 'war', 'animation', 'action', 'drama...",['JP'],1.0,tt0159172,7.9,2281.0,41.250,8.1,2
2,ts35087,Galaxy Express 999,SHOW,Galaxy Express 999 is a manga written and draw...,1978,TV-PG,24,"['scifi', 'action', 'drama', 'animation']",['JP'],1.0,tt0185066,7.8,459.0,5.064,8.3,3
3,ts82387,Lupin the 3rd,SHOW,The master thief Lupin III and his cronies con...,1977,TV-14,24,"['scifi', 'animation', 'action', 'comedy', 'cr...",['JP'],4.0,tt0159192,8.0,1738.0,0.600,NaN,4
4,ts2846,Chōdenji Machine Voltes V,SHOW,Chōdenji Machine Voltes V is a Japanese anime ...,1977,TV-PG,24,"['drama', 'action', 'scifi', 'animation']",['JP'],1.0,tt0159855,8.1,207.0,12.625,7.3,5


In [4]:
#  last 5 data from datset
anime_data.tail()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,index
1076,ts252949,"Farewell, My Dear Cramer",SHOW,With no soccer accomplishments to speak of dur...,2021,TV-14,23,"['comedy', 'drama']",['JP'],1.0,NaN,NaN,NaN,12.235,3.6,1077
1077,ts267874,I★Chu: Halfway Through the Idol,SHOW,Youths aiming to become full-fledged idols enr...,2021,TV-14,24,"['comedy', 'animation']",['JP'],1.0,tt13415416,6.5,24.0,6.699,7.0,1078
1078,ts268254,ABCiee Working Diary,SHOW,"ABCiee is a bit klutzy and ditsy, but he's ful...",2021,NaN,2,['comedy'],['JP'],1.0,tt13985984,5.6,11.0,3.696,5.0,1079
1079,ts285889,GLOOMY The Naughty Grizzly,SHOW,"Gloomy, an abandoned little bear, is rescued b...",2021,NaN,1,"['comedy', 'animation']",['JP'],1.0,tt14827932,4.0,19.0,1.878,3.0,1080
1080,ts255190,Idolls!,SHOW,Go for Hoi Hoi Hall! This is the passionate bu...,2021,TV-14,8,"['comedy', 'animation', 'music']",['JP'],1.0,tt13592988,4.7,15.0,8.032,2.0,1081


In [6]:
anime_data.shape

(1081, 16)

In [8]:
anime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1081 entries, 0 to 1080
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    1081 non-null   object 
 1   title                 1081 non-null   object 
 2   type                  1081 non-null   object 
 3   description           1050 non-null   object 
 4   release_year          1081 non-null   int64  
 5   age_certification     750 non-null    object 
 6   runtime               1081 non-null   int64  
 7   genres                1081 non-null   object 
 8   production_countries  1081 non-null   object 
 9   seasons               1080 non-null   float64
 10  imdb_id               927 non-null    object 
 11  imdb_score            907 non-null    float64
 12  imdb_votes            907 non-null    float64
 13  tmdb_popularity       1068 non-null   float64
 14  tmdb_score            1006 non-null   float64
 15  index                

In [9]:
# Selecting features
anime_features = ["title", "description", "genres"]
print(anime_features)

['title', 'description', 'genres']


In [11]:
for features in anime_features:
  anime_data[features] = anime_data[features].fillna('')

In [12]:
# Combining all features
combining_anime_features = anime_data['title']+' '+anime_data['description']+' '+anime_data['genres']
print(combining_anime_features)

0       Lupin the Third Arsene Lupin III is the grands...
1       Mobile Suit Gundam Mobile Suit Gundam is a tel...
2       Galaxy Express 999 Galaxy Express 999 is a man...
3       Lupin the 3rd The master thief Lupin III and h...
4       Chōdenji Machine Voltes V Chōdenji Machine Vol...
                              ...                        
1076    Farewell, My Dear Cramer With no soccer accomp...
1077    I★Chu: Halfway Through the Idol Youths aiming ...
1078    ABCiee Working Diary ABCiee is a bit klutzy an...
1079    GLOOMY The Naughty Grizzly Gloomy, an abandone...
1080    Idolls! Go for Hoi Hoi Hall! This is the passi...
Length: 1081, dtype: object


In [13]:
vectorizer = TfidfVectorizer()

In [14]:
#  converting features in number
feature_vectors = vectorizer.fit_transform(combining_anime_features)
print(feature_vectors)

  (0, 2223)	0.10681523621720632
  (0, 1920)	0.04001443517750529
  (0, 273)	0.044818246538843844
  (0, 604)	0.027510343974516644
  (0, 8842)	0.04461093070746824
  (0, 11547)	0.1681438634611367
  (0, 5627)	0.17804179078266624
  (0, 4987)	0.15567394978199967
  (0, 4111)	0.1681438634611367
  (0, 3206)	0.14746022762147082
  (0, 536)	0.10681523621720632
  (0, 11186)	0.083287801850836
  (0, 10333)	0.07521831226601851
  (0, 500)	0.06860590455043375
  (0, 4449)	0.17804179078266624
  (0, 4123)	0.13053960910433998
  (0, 7154)	0.09542445038909413
  (0, 7972)	0.1681438634611367
  (0, 4399)	0.05210258324484977
  (0, 6494)	0.16112117226553543
  (0, 3775)	0.17804179078266624
  (0, 5029)	0.13234868048349252
  (0, 6054)	0.08538899429578388
  (0, 11335)	0.17804179078266624
  (0, 5122)	0.16112117226553543
  :	:
  (1080, 4771)	0.24423019669830365
  (1080, 7484)	0.2074416500512196
  (1080, 8874)	0.22101920802800684
  (1080, 8276)	0.21354693866492605
  (1080, 4266)	0.46130527744303285
  (1080, 9751)	0.202279

In [17]:
# Similarities in features
similarity = cosine_similarity(feature_vectors)
# print(similarity)
print(similarity.shape)

(1081, 1081)


In [25]:
anime_name = input("Enter anime name to get suggestion: ")

Enter anime name to get suggestion: Lupin 


In [26]:
#  List of all titles in data
list_of_all_anime_titles = anime_data['title'].tolist()
print(list_of_all_anime_titles)

['Lupin the Third', 'Mobile Suit Gundam', 'Galaxy Express 999', 'Lupin the 3rd', 'Chōdenji Machine Voltes V', 'Space Pirate Captain Harlock', 'The Wonderful Adventures of Nils', 'Princess Knight', 'The Adventures of the Little Prince', 'Space Warrior Baldios', 'Chargeman Ken!', 'Demian, o Justiceiro', 'Dragon Ball Z', 'Dragon Ball', 'Fist of the North Star', 'Megabeast Investigator Juspion', 'Samurai Pizza Cats', 'Gunbuster', 'City Hunter', 'Kimagure Orange Road', "Cat's Eye", 'Dallos', 'Arcadia of My Youth: Endless Orbit SSX', 'Giant Gorg', 'GoShogun', 'Miss Machiko', 'Osomatsu-kun', 'The Wonderful Wizard of Oz', 'Robot Carnival', 'Psycho Armor Govarian', 'Dancougar – Super Beast Machine God', 'Six God Combination Godmars', 'One Piece', 'Cowboy Bebop', 'FLCL', 'Yu Yu Hakusho', 'Ghost Stories', 'Yu-Gi-Oh!', 'InuYasha', 'Mobile Suit Gundam Wing', 'Rurouni Kenshin', 'Case Closed', 'Fighting Spirit', 'Dragon Ball GT', 'Mobile Fighter G Gundam', 'Revolutionary Girl Utena', 'Tenchi Muyo!', 

In [27]:
# Searching for closest match
finding_close_match = difflib.get_close_matches(anime_name, list_of_all_anime_titles)
print(finding_close_match)

['Lupin the 3rd']


In [28]:
close_match = finding_close_match[0]
print(close_match)

Lupin the 3rd


In [30]:
# Index number of close match
index_of_the_anime = anime_data[anime_data.title == close_match]['index'].values[0]
print(index_of_the_anime)

4


In [31]:
#  Similarity score
similarity_score = list(enumerate(similarity[index_of_the_anime]))
print(similarity_score)
len(similarity_score)

[(0, 0.033487397290090724), (1, 0.19316051758267344), (2, 0.07914046144539058), (3, 0.023457517517651915), (4, 1.0), (5, 0.008375183888552708), (6, 0.12565445258847716), (7, 0.04643794739101337), (8, 0.10474965209533801), (9, 0.13587255127090891), (10, 0.06497437326923004), (11, 0.0), (12, 0.1430641088341283), (13, 0.026328450520046114), (14, 0.04635489822589898), (15, 0.06947984253389511), (16, 0.060165028007493626), (17, 0.0710307853015815), (18, 0.009452462668379069), (19, 0.026971075051814546), (20, 0.029316379336344935), (21, 0.018271202419503897), (22, 0.12783839051965082), (23, 0.050132147041888564), (24, 0.103134093622697), (25, 0.01417335717254778), (26, 0.11819299071248755), (27, 0.19097638365211406), (28, 0.08212151559018654), (29, 0.03662308926771908), (30, 0.22878983289158913), (31, 0.07412477527051692), (32, 0.06948001799330207), (33, 0.03142898106519535), (34, 0.014586982740185996), (35, 0.04500814305997522), (36, 0.03776798356626613), (37, 0.02600130385263267), (38, 0.0

1081

In [33]:
sorted_similar_anime = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_anime)

[(4, 1.0), (30, 0.22878983289158913), (1, 0.19316051758267344), (27, 0.19097638365211406), (212, 0.18720921412041838), (415, 0.1765420281031137), (82, 0.16514402583105803), (171, 0.15126717806348788), (72, 0.15107534117175359), (301, 0.1488581227153571), (78, 0.1435655395626538), (12, 0.1430641088341283), (153, 0.13853489153669313), (162, 0.13716838034588097), (9, 0.13587255127090891), (422, 0.1352995093681633), (205, 0.13427815757752273), (130, 0.1340680440212907), (1007, 0.13135920046305508), (22, 0.12783839051965082), (6, 0.12565445258847716), (281, 0.11886620903523588), (26, 0.11819299071248755), (215, 0.11818311874817164), (238, 0.11693858408687276), (211, 0.11636545863764555), (173, 0.11385618849900378), (155, 0.11222973298156916), (104, 0.10842484066556521), (195, 0.10688420722063126), (225, 0.10513050808428437), (8, 0.10474965209533801), (269, 0.10466328430792955), (270, 0.1034879071417139), (24, 0.103134093622697), (432, 0.10270991323984952), (251, 0.10210990286094308), (385, 

In [35]:
print("These are some suggestion for your selected movie: \n")

i = 1

for anime in sorted_similar_anime:
  index = anime[0]
  title_from_index = anime_data[anime_data.index==index]['title'].values[0]
  if (i<31):
    print(i, '.',title_from_index)
    i+=1

These are some suggestion for your selected movie: 

1 . Chōdenji Machine Voltes V
2 . Dancougar – Super Beast Machine God
3 . Mobile Suit Gundam
4 . The Wonderful Wizard of Oz
5 . Air Master
6 . Straight Title Robot Anime
7 . Eat-Man
8 . True Tears
9 . Dai-Guard
10 . gdgd Fairies
11 . VS Knight Lamune & 40 Fire
12 . Dragon Ball Z
13 . Kekkaishi
14 . Tetsujin 28
15 . Space Warrior Baldios
16 . Robot Girls Z
17 . Moyashimon
18 . Digimon Fusion
19 . Digimon Ghost Game
20 . Arcadia of My Youth: Endless Orbit SSX
21 . The Wonderful Adventures of Nils
22 . Brave 10
23 . Osomatsu-kun
24 . Hyakko
25 . Mon Colle Knights
26 . リルぷりっ
27 . Heroman
28 . Mazinger Edition Z: The Impact!
29 . Beyblade: Metal Fusion
30 . Tales of Phantasia: The Animation


In [41]:
anime_name = input("Enter Anime name to get suggestion: ")

list_of_all_anime_titles = anime_data['title'].tolist()

finding_close_match = difflib.get_close_matches(anime_name, list_of_all_anime_titles)

close_match = finding_close_match[0]

index_of_the_anime = anime_data[anime_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_anime]))

sorted_similar_anime = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print("These are some suggestion for your selected Anime: \n")

i = 1

for anime in sorted_similar_anime:
  index = anime[0]
  title_from_index = anime_data[anime_data.index==index]['title'].values[0]
  if (i<31):
    print(i, '.',title_from_index)
    i+=1

Enter Anime name to get suggestion: Dragon ball
These are some suggestion for your selected Anime: 

1 . Fist of the North Star
2 . DD Fist of the North Star
3 . Stellvia of the Universe
4 . The Faraway Paladin
5 . The Asterisk War
6 . Dr. Stone
7 . Assassination Classroom
8 . Somali and the Forest Spirit
9 . Chi's Sweet Home
10 . The Irresponsible Galaxy Tylor
11 . Arpeggio of Blue Steel: Ars Nova
12 . Fist of the Blue Sky: Regenesis
13 . To the Abandoned Sacred Beasts
14 . High School Prodigies Have It Easy Even in Another World!
15 . Twin Star Exorcists
16 . Xuan Yuan Sword Luminary
17 . Conception
18 . Attack on Titan
19 . WorldEnd: What are you doing at the end of the world? Are you busy? Will you save us?
20 . HAKYU HOSHIN ENGI
21 . Saiyuki Reload Blast
22 . Air Master
23 . The World's Finest Assassin Gets Reincarnated in Another World as an Aristocrat
24 . Soul Hunter
25 . Muv-Luv Alternative: Total Eclipse
26 . Burning Kabaddi
27 . Blassreiter
28 . Arifureta: From Commonplace t